In [40]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier,Pool

In [41]:
df=pd.read_csv('loan_data.csv')

In [42]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [43]:
X=df.drop(columns=['loan_status'],axis=1)

le=LabelEncoder()
y=le.fit_transform(df['loan_status'])

In [45]:
cat_columns= X.select_dtypes(include=['object']).columns.tolist()
num_columns = X.select_dtypes(exclude=['object']).columns.tolist()

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [47]:
train_pool=Pool(X_train,y_train,cat_features=cat_columns)

In [50]:
catboost_model= CatBoostClassifier(verbose=0)

param_grid = {
    'depth': [3,4,6,5],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100,200]
}

In [51]:
catboost_model.grid_search(param_grid,X=train_pool,cv=5, stratified=True)


bestTest = 0.2510510795
bestIteration = 99

0:	loss: 0.2510511	best: 0.2510511 (0)	total: 1.68s	remaining: 38.6s

bestTest = 0.190534577
bestIteration = 99

1:	loss: 0.1905346	best: 0.1905346 (1)	total: 3.32s	remaining: 36.5s

bestTest = 0.1763417042
bestIteration = 99

2:	loss: 0.1763417	best: 0.1763417 (2)	total: 4.96s	remaining: 34.7s

bestTest = 0.2169151205
bestIteration = 199

3:	loss: 0.2169151	best: 0.1763417 (2)	total: 8.45s	remaining: 42.2s

bestTest = 0.1762520444
bestIteration = 199

4:	loss: 0.1762520	best: 0.1762520 (4)	total: 12.1s	remaining: 45.9s

bestTest = 0.1674549534
bestIteration = 197

5:	loss: 0.1674550	best: 0.1674550 (5)	total: 15.5s	remaining: 46.6s

bestTest = 0.2358690535
bestIteration = 99

6:	loss: 0.2358691	best: 0.1674550 (5)	total: 17.4s	remaining: 42.3s

bestTest = 0.1831756113
bestIteration = 99

7:	loss: 0.1831756	best: 0.1674550 (5)	total: 19.3s	remaining: 38.6s

bestTest = 0.1728525328
bestIteration = 98

8:	loss: 0.1728525	best: 0.1674550 (5)	to

{'params': {'depth': 6, 'learning_rate': 0.1, 'iterations': 200},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
    

In [52]:
y_pred=catboost_model.predict(X_test)

In [69]:
df_test=X_test.copy()

In [70]:
df_test["loan_status"] = y_test
df_test["predicted_loan_status"] = le.inverse_transform(y_pred)

In [ ]:
df_test